# Streaming

Streaming...

There are two types frequently used with Agents:


In [4]:
from langchain.agents import create_agent
from langchain.chat_models import init_chat_model

In [5]:
model = init_chat_model("gpt-5-mini", model_provider="openai")

agent = create_agent(
    model=model,
    tools=[],
    system_prompt="You are a full-stack comedian",
)

## No Steaming (invoke)

In [6]:
result = agent.invoke({"messages": [{"role": "user", "content": "Tell me a joke"}]})
print(result["messages"][1].content)

Why do programmers prefer dark mode?

Because light attracts bugs.

Want another one?


## values

In [7]:
# Stream = values
for step in agent.stream(
    {"messages": [{"role": "user", "content": "Tell me a joke"}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Tell me a joke
================================== Ai Message ==================================

Why did the full-stack developer go broke?

Because he used up all his cache. 

Want another one?


## message

In [8]:
for token, metadata in agent.stream(
    {"messages": [{"role": "user", "content": "Tell me a joke."}]},
    stream_mode="messages",
):
    print(f"node: {metadata['langgraph_node']}")
    print(f"content: {token.content}")

node: model
content: 
node: model
content: Why
node: model
content:  do
node: model
content:  programmers
node: model
content:  prefer
node: model
content:  dark
node: model
content:  mode
node: model
content: ?
node: model
content:  Because
node: model
content:  light
node: model
content:  attracts
node: model
content:  bugs
node: model
content: .
node: model
content: 


In [9]:
for token, metadata in agent.stream(
    {"messages": [{"role": "user", "content": "Write me a poem."}]},
    stream_mode="messages",
):
    print(f"{token.content}", end="")

I write in caffeine, commit in sleep,
push mornings to a branch where bugs creep.
My heart runs on semicolons and late-night tabs,
console.log confessions, tiny unwrapped jabs.

The server hums like a distant choir,
404s like lost socks, retries on wire.
I stitch together pages with duct-tape code,
and pray the stylesheet forgives what I broke.

You arrive as a viewport, resized and new,
responsive in smile, graceful in hue.
We cache quiet moments, expire old pain,
refresh for laughter, GET love again.

If life throws exceptions, I’ll catch with a grin—
deploy a new version: we begin again.

## Tools can stream too!

In [10]:
from langchain.agents import create_agent
from langgraph.config import get_stream_writer


def get_weather(city: str) -> str:
    """Get weather for a given city."""
    writer = get_stream_writer()
    # stream any arbitrary data
    writer(f"Looking up data for city: {city}")
    writer(f"Acquired data for city: {city}")
    return f"It's always sunny in {city}!"


agent = create_agent(
    model="openai:gpt-5-mini",
    tools=[get_weather],
)

for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode=["updates", "custom"],
):
    print(chunk)

('updates', {'model': {'thread_model_call_count': 1, 'run_model_call_count': 1, 'messages': [AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 88, 'prompt_tokens': 132, 'total_tokens': 220, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 64, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CNnmt0XAk1DNhRy2vJ7pyM4dONEfq', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--f680c744-5ef7-4887-917c-ca2a20609cab-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'San Francisco'}, 'id': 'call_cbNoBFsPtTGQUAEdEaadFfP6', 'type': 'tool_call'}], usage_metadata={'input_tokens': 132, 'output_tokens': 88, 'total_tokens': 220, 'input_token_details': {'audio': 0, 'cache_read': 0},

In [11]:
for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode=["updates", "custom"],
):
    if chunk[0] == "custom":
        print(chunk[1])

Looking up data for city: San Francisco
Acquired data for city: San Francisco
